In [1]:
import torch
from dataset import AudioTrainDataset, AudioTestDataset
from torch.utils.data import DataLoader
from model import NN 
from config import Train_config
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

random.seed(2021)

In [2]:
config = Train_config()

In [3]:
index = [i for i in range(2000)]
random.shuffle(index)
train_index = index[:1800]
test_index =  index[1800:]

In [4]:
train = AudioTrainDataset(config.train_data_dir, train_index)
test = AudioTrainDataset(config.train_data_dir, test_index)

In [5]:
trainloader = DataLoader(train, batch_size = config.train_batch_size, shuffle = True)
testloader = DataLoader(test, batch_size = config.test_batch_size, shuffle = False)

In [6]:
device = torch.device(config.device)

In [7]:
model = NN().to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = config.lr)

In [9]:
max_test_acc = 0
for epoch in range(config.max_epoch):
    model.train()
    total_loss = []
    total_acc = []
    for data in tqdm(trainloader):
        data_input,label = data

        data_input = data_input.to(torch.float32).to(device)
        label = label.to(device)
        output = model(data_input)
        loss = criterion(output,label)
        total_loss.append(loss.data.cpu().numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_ = output.data.cpu().numpy()
        y_ = np.argmax(y_,axis = 1)
        label = label.data.cpu().numpy()
        total_acc.append(accuracy_score(label, y_))
    train_loss = np.mean(total_loss)
    train_acc = np.mean(total_acc)

    model.eval()    
    total_loss = []
    total_acc = []
    total_precision = []
    total_recall = []
    total_f1 = []
    for data in testloader:
        data_input,label = data
        data_input = data_input.to(torch.float32).to(device)
        label = label.to(device)
        output = model(data_input)
        loss = criterion(output,label).cpu().detach().numpy()
        total_loss.append(loss)
        y_ = output.data.cpu().numpy()
        y_ = np.argmax(y_,axis = 1)
        label = label.data.cpu().numpy()
        total_precision.append(precision_score(label, y_))
        total_recall.append(recall_score(label, y_))
        total_f1.append(f1_score(label, y_))
        total_acc.append(accuracy_score(label, y_))
    test_loss = np.mean(total_loss)
    test_acc = np.mean(total_acc)
    test_precision = np.mean(total_precision)
    test_recall = np.mean(total_recall)
    test_f1 = np.mean(total_f1)

    if test_acc > max_test_acc:
        torch.save(model.state_dict(), config.save_path)
        max_test_acc = test_acc
    
    print(f"Epoch {epoch+1}, train loss is {train_loss:.4f}, train acc is {train_acc:.4f}, test loss is {test_loss:.4f}, test acc is {test_acc:.4f},")
    print(f"test precision is {test_precision:.4f}, test recall is {test_recall:.4f}, test f1 is {test_f1:.4f}.")

  2%|▏         | 1/57 [00:00<00:08,  6.78it/s]

Epoch 1, train loss is 0.6193, train acc is 0.6946, test loss is 0.5733, test acc is 0.7400,
test precision is 0.7400, test recall is 1.0000, test f1 is 0.8506.


  2%|▏         | 1/57 [00:00<00:10,  5.57it/s]

Epoch 2, train loss is 0.6137, train acc is 0.6996, test loss is 0.5733, test acc is 0.7400,
test precision is 0.7400, test recall is 1.0000, test f1 is 0.8506.


  2%|▏         | 1/57 [00:00<00:10,  5.14it/s]

...

Epoch 48, train loss is 0.3337, train acc is 0.9792, test loss is 0.3737, test acc is 0.9350,
test precision is 0.9245, test recall is 0.9932, test f1 is 0.9577.


  2%|▏         | 1/57 [00:00<00:08,  6.63it/s]

Epoch 49, train loss is 0.3342, train acc is 0.9786, test loss is 0.3522, test acc is 0.9600,
test precision is 0.9730, test recall is 0.9730, test f1 is 0.9730.


100%|██████████| 57/57 [00:08<00:00,  6.53it/s]


Epoch 50, train loss is 0.3303, train acc is 0.9830, test loss is 0.3551, test acc is 0.9600,
test precision is 0.9545, test recall is 0.9932, test f1 is 0.9735.


In [10]:
model.load_state_dict(torch.load(config.save_dir))

<All keys matched successfully>

In [11]:
model.eval()    
total_loss = []
total_acc = []
total_precision = []
total_recall = []
total_f1 = []
total_y = []
total_label = []
for data in testloader:
    data_input,label = data
    data_input = data_input.to(torch.float32).to(device)
    label = label.to(device)
    output = model(data_input)
    loss = criterion(output,label).cpu().detach().numpy()
    total_loss.append(loss)
    y_ = output.data.cpu().numpy()
    y_ = np.argmax(y_,axis = 1)
    label = label.data.cpu().numpy()
    total_precision.append(precision_score(label, y_))
    total_recall.append(recall_score(label, y_))
    total_f1.append(f1_score(label, y_))
    total_acc.append(accuracy_score(label, y_))
    total_y.extend(list(y_))
    total_label.extend(list(label))
test_loss = np.mean(total_loss)
test_acc = np.mean(total_acc)
test_precision = np.mean(total_precision)
test_recall = np.mean(total_recall)
test_f1 = np.mean(total_f1)
print(f"test loss is {test_loss:.4f}, test acc is {test_acc:.4f}, test precision is {test_precision:.4f}, test recall is {test_recall:.4f}, test f1 is {test_f1:.4f}.")
print("test confusion matrix:")
print(confusion_matrix(total_y, total_label))

test loss is 0.3454, test acc is 0.9700, test precision is 0.9863, test recall is 0.9730, test f1 is 0.9796.
test confusion matrix:
[[ 50   4]
 [  2 144]]


In [12]:
test = AudioTestDataset(config.test_data_dir)

In [13]:
testloader = DataLoader(test, batch_size = config.test_batch_size, shuffle=False)

In [14]:
model.eval()
total_y = []
df = pd.read_excel(config.test_data_dir)
filename = list(df['filename'])
bf = list(df['bf'])
for data in testloader:
    data = data.to(torch.float32).to(device)
    output = model(data)
    y_ = output.data.cpu().numpy()
    y_ = np.argmax(y_,axis = 1)
    total_y.extend(list(y_))
l = []
for i in range(df.shape[0]):
    l.append([filename[i],total_y[i], bf[i]])

In [15]:
res = pd.DataFrame(l, columns=['filename', 'label', 'bf'])
res.to_excel(config.test_data_dir, index=None)

In [16]:
df = pd.read_excel(config.test_fill_dir)
filename = list(df['filename'])
l = []
for i in range(df.shape[0]):
    l.append([filename[i],total_y[i]])

In [17]:
res = pd.DataFrame(l, columns=['filename', 'label'])
res.to_excel(config.test_fill_dir, index=None)